In [14]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp


In [15]:
mp_holistic = mp.solutions.holistic #holistic model
mp_drawing = mp.solutions.drawing_utils # drawing utilities

In [16]:
def draw_styled_landmark(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), #dot color
                mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)) # outline Color
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(0,255,255), thickness=2, circle_radius=2))
     
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

In [17]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False # Image is no longer writeable
    results = model.process(image) #make prediction
    image.flags.writeable = True # image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

<h3> 3. Extract Keypoint</h3>

In [18]:
def extract_keypoints(results):
    # if didnt detect any face create np zeros 33*4 , 33 because pose landmark is 33 and times 4 because it has 4 variable, X, Y, Z and Visibility the other dont have Visibility
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # if didnt detect any face create np zeros 468*3 , 468 because face landmark is 21
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    # if didnt detect any (left/right) hand create np zeros 21*3 , 21 because hands landmark is 21
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face, lh, rh])


4. Setup Folder For Collection

In [35]:
#path for exported data
DATA_PATH = os.path.join('MP_3Data')
#action to detect
# JSL
# Link: https://www.youtube.com/watch?v=KmZs2uloCEc&ab_channel=DeafJapanTV
# https://youtu.be/PSqxz-OmT-E?si=DohA5-U29HlAk37q
actions = np.array(['hello', 'thanks','iloveyou','good','seeyoulater', 'learnit','whatisit', 'youarepretty','ithink','itsnothing'])
# actions = np.array(['seeyoulater' ])
# 30 video worth of data
no_sequences = 50
# video are going to be 20 frames in length
sequence_length = 20

In [20]:
# Hello
# 0
# 1
# 2
# 29
# thanks
#i love you
#f##kyou
# friend
# help

In [36]:
for action in actions:
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass
    print(action)

hello
thanks
iloveyou
good
seeyoulater
learnit
whatisit
youarepretty
ithink
itsnothing


<h3>5. Collect Keypoint Values for Training</h3>

In [40]:
# use this so you can input it one by one
# actions = np.array(['hello', 'thanks','iloveyou','good','seeyoulater', 'learnit','whatisit', 'youarepretty','ithink','itsnothing'])
# actions = np.array(['ithink'])

In [41]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #NEW loop
    #loop through actions
    for action in actions:
        #loop throug sequence aka video
        for sequence in range(no_sequences):
            #loop through video length 
            for frame_num in range(sequence_length):
                
                ret, frame = cap.read()
                frame = cv2.flip(frame, 1)
                #make predictions
                image, results = mediapipe_detection(frame, holistic)
                # applying or draw landmark visual to the current image
                draw_styled_landmark(image, results)
                
                # NEW apply collection logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    # frame of the video,  text, X,Y location of the text, FONT type, scalar text, color font, thickness, 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (20,30), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(1000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (20,30), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                #NEW export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path,keypoints)
                  
                cv2.imshow("OpenCV Feed",image)
                
                if cv2.waitKey(10) & 0XFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()
cap.release()
cv2.destroyAllWindows()

In [39]:
#momentary stop when you interrupt the video
cap.release()
cv2.destroyAllWindows()

In [ ]:
#this is just a briefing for you to see your keypoints
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened:
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        #make predictions
        image, results = mediapipe_detection(frame, holistic)
        #     frame of the video,  text, X,Y location of the text, FONT type, scalar text, color font, thickness, 
        draw_styled_landmark(image, results) # applying or draw landmark visual to the current image
        
        cv2.imshow("OpenCV Feed",image)
        if cv2.waitKey(10) & 0XFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()